[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bsbodden/redis_vss_getting_started/blob/main/vector_similarity_with_redis.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/bsbodden/redis_vss_getting_started/main/assets/nbviewer-shield.svg)](https://nbviewer.org/github/bsbodden/redis_vss_getting_started/blob/main/vector_similarity_with_redis.ipynb)

# Similarity Search with Redis as a Vector Database

## The "Unstructured Data" Problem

Today, about 80% of the data organizations generate is "unstructured"; data that either does not have a well-defined schema or cannot be restructured into a familiar columnar format. Typical examples of unstructured data include free-form text, images, videos, and sound clips. Moreover, this data imbalance towards unstructured data is expected to grow in the coming decades.

Unstructured data is high-dimensional and noisy,    making it more challenging to analyze and interpret using traditional methods. But it is also packed with information and meaning.  

Traditionally, unstructured data is processed to extract specific features, effectively turning it into structured data. Once in the realm of structured data, we can search the data with SQL queries (if stored in a relational database) or with a text search engine. 
The approach of transforming unstructured data into structured data has a few issues; first, engineering features out of unstructured data can be computationally expensive and error-prone, significantly delaying when we can effectively use the data. And secondly, in the extractions/transformation process, lose fidelity and information since unique, 'latent' features are likely lost or overlooked because they can't be easily categorized or quantified.

## Enter "Vector Databases"

An approach to dealing with unstructured data is to "vectorize" such data. By "vectorizing," we mean to somehow convert something like a text passage, an image, a video, or a song into a flat sequence of numbers representing a particular piece of data. These "vectors" are representations of the data in N-dimensional space. By vectorizing, we gain the ability to use linear algebra techniques to compare, group, and operate on our data. This is the foundation of a Vector Database; the ability to store and operate on vectors.
This approach is not new and has been around for eons. The difference today is how the techniques for generating the vectors have advanced.

## Using Machine Learning "Embeddings" as Vectors

Traditional methods for converting unstructured data into vector form include Bag-of-Words (BoW) and TF-IDF (Term Frequency-Inverse Document Frequency) for textual data. For categorical data, one-hot Encoding is a commonly used approach. And Hashing and Feature Extraction techniques, such as edge detection, texture analysis, or color histograms, have been employed for high-dimensionality data like images.

While powerful in their own right, these approaches reveal limitations when confronted with high-dimensional and intricate data forms like long text passages, images, and audio. Consider, for example, how a text passage could be restructured—through sentence rearrangement, synonym usage, or alterations in narrative style. Such Simple modifications could effectively sidestep techniques like Bag-of-Words, preventing systems using the generated encodings from identifying texts with similar meanings.

This is where advancements in Machine Learning, particularly Deep Learning, make their mark. Machine Learning models have facilitated the rise of 'embeddings' as a widely embraced method for generating dense, low-dimensional vector representations. Given a suitable model for the task at hand, the generated embeddings can encapsulate complex patterns and semantic meanings inherent in data, thus overcoming the limitations of their traditional counterparts.

### Generating Embeddings for the "Bikes" Dataset

To investigate Vector Similarity, we'll use a subset of the Bikes dataset, a relatively simple synthetic dataset. The dataset has 11 bicycle records in a JSON file named `bikes.json` and includes the fields `model`, `brand`, `price`, `type`, `specs`, and `description`. The `description` field is particularly interesting for our purposes since it consists of a free-form textual description of a bicycle.

#### Loading JSON "Bikes" Dataset

Let's load the bikes dataset as a JSON array:

In [ ]:
!pwd
!pip install --upgrade pip

# Install required libraries
!python3 -m pip -q install boto3 redis pandas matplotlib

#%pip install -q redis pandas matplotlib
#%pip -q install pandas
#%pip install matplotlib

#Install `sentence-transformers` library -- This is optional if you are using AWS Bedrock
%pip install -U -q sentence-transformers

In [ ]:
import requests
import json

url = 'https://raw.githubusercontent.com/bsbodden/redis_vss_getting_started/main/data/bikes.json'
response = requests.get(url)
bikes = json.loads(response.text)

#### Inspect the Bikes JSON

Let's inspect the content of the JSON array in table form using the Pandas framework:

In [ ]:
import pandas as pd

pd.DataFrame(bikes)

Let's take a look at the structure of one of our bike JSON documents:

In [ ]:
print(json.dumps(bikes[0], indent=2))

#### Generating Text Embeddings using SentenceTransformers

We will use the [SentenceTransformers](https://www.sbert.net/) framework to generate embeddings for the bikes descriptions. Sentence-BERT (SBERT) is a BERT model modification that produces consistent and contextually rich sentence embeddings. SBERT improves tasks like semantic search and text grouping by allowing for efficient and meaningful comparison of sentence-level semantic similarity.


### Selecting a suitable pre-trained Model

We must pick a suitable model based on the task at hand when generating embeddings. In our case, we want to query for bicycles using short sentences against the longer bicycle descriptions. This is referred to as "Asymmetric Semantic Search," often employed in cases where the search query and the documents being searched are of different nature or structure. Suitable models for asymmetric semantic search include pre-trained [MS MARCO](https://microsoft.github.io/msmarco/) Models. MS MARCO models are trained on the **M**icro**S**oft **MA**chine **R**eading **CO**mprehension dataset, and are optimized for understanding real-world queries and retrieving relevant responses. They are widely used in search engines, chatbots, and other AI applications. At the time of this writing, the highest performing MS MARCO model tuned for cosine-similarity available from SentenceTranformers is `msmarco-distilbert-base-v4`. 


In [ ]:
from textwrap import TextWrapper

sample_description = bikes[0]['description']
TextWrapper(width=120).wrap(sample_description)

## Storing our bikes in Redis
Now that we know how to vectorize the bikes descriptions, it's time to start working with Redis. We will use Redis Enterprise Cloud database to store our vectors. 
Note: In case Redis Enterprise is not available, use Redis Stack open source


In [ ]:
# Uncomment & execute the following code in case Redis Enterprise is not available
# !curl -fsSL https://packages.redis.io/redis-stack/redis-stack-server-6.2.6-v7.focal.x86_64.tar.gz -o redis-stack-server.tar.gz
# !tar -xvf redis-stack-server.tar.gz
# !./redis-stack-server-6.2.6-v7/bin/redis-stack-server --daemonize yes
#oss = True

### Note down the parameters required for Redis Enterprise Cloud like URL, password, port etc

In [ ]:
host = 'redis-15315.c212.ap-south-1-1.ec2.cloud.redislabs.com'
port = 15315
password = 'admin'
requirePass = True

### Redis Python Client

To interact with Redis, we'll install the [`redis-py`](https://github.com/redis/redis-py) client library, which encapsulates the commands to work with OSS Redis as well as Redis Stack:

#### Create a `redis-py` client and test the server

We'll instantiate the Redis client, connecting to the localhost on Redis' default port `6379`. By default, Redis returns binary responses; to decode them, we'll pass the `decode_responses` parameter set to `True`:

In [ ]:
import redis

if requirePass:
    client = redis.Redis(host = host, port=port, decode_responses=True, password=password)
else:
    client = redis.Redis(host = 'localhost', decode_responses=True)


Let's use Redis' [`PING`](https://redis.io/commands/ping/) command to check that Redis is up and running:

In [ ]:
client.ping()


### Storing the Bikes as JSON Documents in Redis

Redis Stack includes [JSON](https://redis.io/docs/stack/json/) functionality. Like any other Redis data type, the JSON datatype allows you to use Redis commands to save, update, and retrieve JSON values. 
Since we already have the bikes data loaded in memory as the `bikes` JSON array. We will iterate over `bikes`, generate a suitable Redis key and store them in Redis using the [`JSON.SET`](https://redis.io/commands/json.set/) command. We'll do this [pipeline](https://redis.io/docs/manual/pipelining/) mode to minimize the round-trip times:

In [ ]:
pipeline = client.pipeline()

for i, bike in enumerate(bikes, start=1):
    redis_key = f'bikes:{i:03}'
    print(redis_key)
    pipeline.json().set(redis_key, '$', bike)

pipeline.execute()

Let's retrieve a specific value from one of the JSON bikes in Redis using a [JSONPath](https://goessner.net/articles/JsonPath/) expression:

In [ ]:
client.json().get('bikes:010', '$.model')

### Vectorize all of the Bikes Descriptions

To vectorize all the descriptions in the database, we will first collect all the Redis keys for the bikes:

We'll use the keys as a parameter to the [`JSON.MGET`](https://redis.io/commands/json.mget/) command, along with the JSONPath expression `$.description` to collect the descriptions in a list which we will then pass to the `encode` method to get a list of vectorized embeddings:

In [ ]:
import numpy as np

# we will first collect all the Redis keys for the bikes
keys = sorted(client.keys('bikes:*'))

# Next will accumulate all the descriptions in List format
descriptions = client.json().mget(keys, '$.description')
descriptions = [item for sublist in descriptions for item in sublist]


## To create the vector embeddings we can leverage:
1. Recommended: Use AWS Bedrock API and invoke Titan embedding model
2. Another option: Use the SentenceTransformers framework to generate embeddings for the bikes descriptions.
   Sentence-BERT (SBERT) is a BERT model modification that produces consistent and contextually rich sentence embeddings. 
   SBERT improves tasks like semantic search and text grouping by allowing for efficient and meaningful comparison of 
   sentence-level semantic similarity.

## Option 1: Embedding using Bedrock API

Now we can add the vectorized descriptions to the JSON documents in Redis using the `JSON.SET` command to insert a new field in each of the documents under the JSONPath `$.description_embeddings`, once again we'll do this in pipeline mode:

In [ ]:
import boto3
import json

#bedrock2 = boto3.client('bedrock')

# This method that generates embedding using Bedrock API with Titan embedding model
def generate_embedding(body):
    modelId = 'amazon.titan-e1t-medium'
    accept = 'application/json'
    contentType = 'application/json'
    
    bedrock_runtime = boto3.client('bedrock-runtime')
    #bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name="us-west-2")

    response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body.get('embedding')
    return embedding


Let's check if Bedrock embeddings API are working well and take a peek at the first 5 elements of the generated vector:

In [ ]:
body = json.dumps({"inputText": sample_description})
embedding = generate_embedding(body)
print(embedding[:5])

# Also check the dimension of the model
VECTOR_DIMENSION = len(embedding)
VECTOR_DIMENSION

#### Generate & store the embeddings 
If everything is fine at this point, we can proceed further.
Here we will do 2 things:
1. Generate the embeddings of 'description' field present in the JSON object.
2. And finally storing the embeddings in a new element 'description_embeddings' inside the same JSON object

In [ ]:
# Create a pipeline with redis server
pipeline = client.pipeline()

for key, description in zip(keys, descriptions):
    input_query = json.dumps({"inputText": description})
    embedding = generate_embedding(body=input_query)
    pipeline.json().set(key, '$.description_embeddings', embedding)

pipeline.execute()

## Option 2: Embedding using SentenceTransformers BERT API
If option 1 is completed successfully, skip Option 2 and proceed to the next section.
However, if you have issues in following Option 1 (For e.g, due to any issues related to Bedrock APIs like outage, throttling, rate-limiting etc), 
you can follow this option

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('msmarco-distilbert-base-v4')

embedding = embedder.encode(sample_description)
body = json.dumps({"inputText": sample_description})
embedding = generate_embedding(body)

Let's check if SentenceTransformer embeddings API are working well and take a peek at the first 5 elements of the generated vector:

In [ ]:
embedding = embedder.encode(sample_description)
print(embedding[:5])

# Also check the dimension of the model
VECTOR_DIMENSION = len(embedding)
VECTOR_DIMENSION

#### Generate & store the embeddings 
If everything is fine at this point, we can proceed further.
Here we will do 2 things:
1. Generate the embeddings of 'description' field present in the JSON object.
2. And finally storing the embeddings in a new element 'description_embeddings' inside the same JSON object

In [ ]:
pipeline = client.pipeline()

for key, embedding in zip(keys, embeddings):
    pipeline.json().set(key, '$.description_embeddings', embedding)

pipeline.execute()

### Verify the data in Redis
Let's inspect one of the vectorized bike documents using the `JSON.GET` command:

In [ ]:
print(json.dumps(client.json().get('bikes:010'), indent=2)) 

When storing a vector embedding as part of a JSON datatype, the embedding is stored as a JSON array, in our case, under the field `description_embeddings` as shown.

----------------------------------------------------------------------------------------

### Making the bikes collection searchable

Redis Stack provides a powerful search engine ([RediSearch](https://redis.io/docs/stack/search/)) that introduces [commands](https://redis.io/docs/stack/search/commands/) to create and maintain search indices for both collections of HASHES and [JSON](https://redis.io/docs/stack/search/indexing_json/) documents.

To create a search index for the bikes collection, we'll use the [`FT.CREATE`](https://redis.io/commands/ft.create/) command:

```
1️⃣ FT.CREATE idx:bikes_vss ON JSON 
2️⃣  PREFIX 1 bikes: SCORE 1.0 
3️⃣  SCHEMA 
4️⃣    $.model TEXT WEIGHT 1.0 NOSTEM 
5️⃣    $.brand TEXT WEIGHT 1.0 NOSTEM 
6️⃣    $.price NUMERIC 
7️⃣    $.type TAG SEPARATOR "," 
8️⃣    $.description AS description TEXT WEIGHT 1.0 
9️⃣    $.description_embeddings AS vector VECTOR FLAT 6 TYPE FLOAT32 DIM 768 DISTANCE_METRIC COSINE
```

There is a lot to unpack here; let's take it from the top:

- 1️⃣ We start by specifying the name of the index; `idx:bikes` indexing keys of type `JSON`.
- 2️⃣ The keys being indexed are found using the `bikes:` key prefix.
- 3️⃣ The `SCHEMA` keyword marks the beginning of the schema field definitions.
- 4️⃣ Declares that field in the JSON document at the JSONPath `$.model` will be indexed as a `TEXT` field, allowing full-text search queries (disabling stemming).
- 5️⃣ The `$.brand` field will also be treated as a `TEXT` schema field.
- 6️⃣ The `$.price` field will be indexed as a `NUMERIC` allowing numeric range queries.
- 7️⃣ The `$.type` field will be indexed as a `TAG` field. Tag fields allow exact-match queries, and are suitable for categorical values.
- 8️⃣ The `$.description` field will also be indexed as a `TEXT` field.
- 9️⃣ Finally, the vector embeddings in `$.description_embeddings` are indexed as a `VECTOR` field and assigned to the alias `vector`. 
  
Let's break down the `VECTOR` schema field definition to better understand the inner workings of Vector Similarity in Redis:

* `FLAT`: Specifies the indexing method, which can be `FLAT` or `HNSW`. FLAT (brute-force indexing) provides exact results but at a higher computational cost, while HNSW (Hierarchical Navigable Small World) is a more efficient method that provides approximate results with lower computational overhead.
* `TYPE`: Set to `FLOAT32`. Current supported types are `FLOAT32` and `FLOAT64`.
* `DIM`: The length or dimension of our embeddings, which we determined previosly to be `768`.
* `DISTANCE_METRIC`: One of `L2`, `IP`, `COSINE`. 
  - `L2` stands for "Euclidean distance"; a straight-line distance between the vectors. Preferred when the absolute differences, including magnitude, matter most.
  - `IP` stands for "Inner Product"; IP measures the projection of one vector onto another. It emphasizes the angle between vectors rather than their absolute positions in the vector space.
  - `COSINE` stands for "Cosine Similarity"; a normalized form of inner product. This metric measures only the angle between two vectors, making it magnitude-independent.  
  - For our querying purposes, the direction of the vectors carry more meaning (indicating semantic similarity), and the magnitude is largely influenced by the length of the documents, therefore `COSINE` similarity is chosen. Also, our chosen embedding model is fine-tuned for Cosine Similarity.

The Python code below creates the Redis Search Index for the bikes collection equivalent to the previous raw `FT.CREATE` command:

In [ ]:
from redis.commands.search.field import TagField, TextField, NumericField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query

INDEX_NAME = 'idx:bikes_vss'
DOC_PREFIX = 'bikes:'

try:
    # check to see if index exists
    client.ft(INDEX_NAME).info()
    print('Index already exists!')
except:
    # schema
    schema = (
        TextField('$.model', no_stem=True, as_name='model'),
        TextField('$.brand', no_stem=True, as_name='brand'),
        NumericField('$.price', as_name='price'),
        TagField('$.type', as_name='type'),
        TextField('$.description', as_name='description'),
        VectorField('$.description_embeddings',
            'FLAT', {
                'TYPE': 'FLOAT32',
                'DIM': VECTOR_DIMENSION,
                'DISTANCE_METRIC': 'COSINE',
            },  as_name='vector'
        ),
    )

    # index Definition
    definition = IndexDefinition(prefix=[DOC_PREFIX], index_type=IndexType.JSON)

    # create Index
    client.ft(INDEX_NAME).create_index(fields=schema, definition=definition)

#### Check the state of the index

After the `FT.CREATE` creates the index, the indexing process is automatically started in the background. In the blink of an eye, our 11 JSON documents should be indexed and ready to be searched. To corroborate that, we use the [`FT.INFO`](https://redis.io/commands/ft.info/) command to check some information and statistics on the index. Of particular interest are the number of documents successfully indexed, and the number of failures:  

In [ ]:
info = client.ft(INDEX_NAME).info()

num_docs = info['num_docs']
indexing_failures = info['hash_indexing_failures']
total_indexing_time = info['total_indexing_time']
percent_indexed = int(info['percent_indexed']) * 100


print(f"{num_docs} documents ({percent_indexed} percent) indexed with {indexing_failures} failures in {float(total_indexing_time):.2f} milliseconds")

### Structured Data Searches with Redis

The index `idx:bikes_vss` indexes the structured fields of our JSON documents `model`, `brand`, `price`, and `type`. It also indexes the unstructured free-form text `description` and the generated embeddings in `description_embeddings`. Before we dive deeper into Vector Similarity Search, we need to understand the basics of querying a Redis index. The Redis command of interest is [`FT.SEARCH`](https://redis.io/commands/ft.search/). Like a SQL select statement, an `FT.SEARCH` invocation can be as simple or as complex as needed. 

Let's try simple queries that give enough context to complete our VSS examples. For example, to retrieve all bikes where the `brand` is `Peaknetic`, we can use the following command:

```
FT.SEARCH idx:bikes_vss '@brand:Peaknetic'
```

The command will return all matching documents. With the inclusion of the vector embeddings, that's a little too verbose. If we wanted to only return specific fields from our JSON documents, for example, the document `id`, the `brand`, `model` and `price`, we could use:

```
FT.SEARCH idx:bikes_vss '@brand:Peaknetic' RETURN 4 id, brand, model, price
```

In the query, we are searching against a schema field of type `TEXT`. The equivalent Python code is:

In [ ]:
query = (
    Query('@brand:Peaknetic').return_fields('id', 'brand', 'model', 'price')
)
client.ft(INDEX_NAME).search(query).docs

Let's say we only wanted bikes under `$1000`. We can add a numeric range clause to our query since the `price` field is indexed as `NUMERIC`:

In [ ]:
query = (
    Query('@brand:Peaknetic @price:[0 1000]').return_fields('id', 'brand', 'model', 'price')
)
client.ft(INDEX_NAME).search(query).docs

### Semantic Searching with Vector Similarity Search

Now that the bikes collection is stored and properly indexed in Redis, we want to query them using short query prompts. Let's put our queries in a list so we can execute them in bulk:

In [ ]:
queries = [
    'Bike for small kids',
    'Best Mountain bikes for kids',
    'Cheap Mountain bike for kids',
    'Female specific mountain bike',
    'Road bike for beginners',
    'Commuter bike for people over 60',
    'Comfortable commuter bike',
    'Good bike for college students',
    'Mountain bike for beginners',
    'Vintage bike',
    'Comfortable city bike'
]

We need to encode the query prompts to query the database using VSS. Just like we did with the descriptions of the bikes, we'll use the SentenceTransformers model to encode the queries:

In [ ]:
#encoded_queries = embedder.encode(queries)
#len(encoded_queries)
encoded_queries = []
for query in queries:
    input_query = json.dumps({"inputText": query})
    query_embedding = generate_embedding(body=input_query)
    encoded_queries.append(query_embedding)

len(encoded_queries)


#### Constructing a "Pure KNN" VSS Query

We'll start with a K-nearest neighbors (KNN) query. KNN is a foundational algorithm used in vector similarity search, where the goal is to find the most similar items to a given query item. Using the chosen distance metric, the KNN algorithm calculates the distance between the query vector and each vector in the database. It then returns the 'K' items with the smallest distances to the query vector. These are considered to be the most similar items.

The syntax for vector similarity KNN queries is `(*)=>[vector_similarity_query>]` where the `(*)` (the `*` meaning all) is the filter query for the search engine. That way, one can reduce the search space by filtering the collection on which the KNN algorithm operates. 

* The `$query_vector` represents the query parameter we'll use to pass the vectorized query prompt.
* The results will be filtered by `vector_score`, which is a field derived from the name of the field indexed as a Vector by appending `_score` to it, in our case, `vector` (the alias for `$.description_embeddings`). 
* Our query will return the `vector_score`, the `id` of the match documents, and the `$.brand`, `$.model`, and `$.description`. 
* Finally, to utilize a vector similarity query with the `FT.SEARCH` command, we must specify DIALECT 2 or greater.

In [ ]:
query = (
    Query('(*)=>[KNN 3 @vector $query_vector AS vector_score]')
     .sort_by('vector_score')
     .return_fields('vector_score', 'id', 'brand', 'model', 'description')
     .dialect(2)
)

client.ft(INDEX_NAME).search(query, { 'query_vector': np.array(encoded_queries[0], dtype=np.float32).tobytes() }).docs

We pass the vectorized query as `$query_vector` to the search function to execute the query. The following code shows an example of creating a NumPy array from a vectorized query prompt (`encoded_query`) as a single precision float array and converting it into a compact, byte-level representation that we can pass as a Redis parameter:

```python .noeval
client.ft(INDEX_NAME).search(query, { 'query_vector': np.array(encoded_query, dtype=np.float32).tobytes() }).docs
````

With the template for the query in place, we can use a bit of Python code to execute all query prompts in a loop, passing the vectorized query prompts. Notice that for each result we calculate the `vector_score` as `1 - doc.vector_score`, since we use cosine "distance" as the metric, the items with the smallest "distance" are closer and therefore more similar to our query. 

We will then loop over the matched documents and create a list of results that will help up make a lovely Pandas table to visualize the results:

In [ ]:
from IPython.display import display, HTML

def create_query_table(query, queries, encoded_queries, extra_params = {}):
    results_list = []
    for i, encoded_query in enumerate(encoded_queries):
        result_docs = client.ft(INDEX_NAME).search(query, { 'query_vector': np.array(encoded_query, dtype=np.float32).tobytes() } | extra_params).docs
        for doc in result_docs:
            vector_score = round(1 - float(doc.vector_score), 2)
            results_list.append({
                'query': queries[i], 
                'score': vector_score, 
                'id': doc.id,
                'brand': doc.brand,
                'model': doc.model,
                'description': doc.description
            })

    # Pretty-print the table
    queries_table = pd.DataFrame(results_list)
    queries_table.sort_values(by=['query', 'score'], ascending=[True, False], inplace=True)
    queries_table['query'] = queries_table.groupby('query')['query'].transform(lambda x: [x.iloc[0]] + ['']*(len(x)-1))
    queries_table['description'] = queries_table['description'].apply(lambda x: (x[:497] + '...') if len(x) > 500 else x)
    html = queries_table.to_html(index=False)
    display(HTML(html))

The query results show the individual queries' top 3 matches (our K parameter) along with the bike's id, brand, and model for each query. For example, for the query  "Best Mountain bikes for kids", the highest similarity score (`0.54`) and therefore the closest match was the 'Nord' brand 'Chook air 5' bike model, described as:

> "The Chook Air 5 gives kids aged six years and older a durable and uberlight mountain bike for their first experience on tracks and easy cruising through forests and fields. The lower top tube makes it easy to mount and dismount in any situation, giving your kids greater safety on the trails. The Chook Air 5 is the perfect intro to mountain biking."

From the description, we gather that this bike is an excellent match for younger children, and the MS MARCO model-generated embeddings seem to have captured the semantics of the description accurately.

In [ ]:
create_query_table(query, queries, encoded_queries)

Sometimes "a picture is worth a thousand words". Using the dimensionality reduction technique [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding) we can create a 3-d representation of our description embeddings, as well as the query embeddings which shows how well the MS Marco Sentence Embeddings clustered our bicycle descriptions and we can visually judge a query's distance to a specific bike:

![t-SNE 3-D Embeddings Visualization](https://raw.githubusercontent.com/bsbodden/redis_vss_getting_started/3ac967dfbdd84dad25ade620826c0e01ac0251ca/embeddings-tsne.png)

#### Hybrid Queries

"Pure KNN" queries, as in the previous section, evaluate a query against the whole space of vectors in a data collection. The larger the collection, the more computationally expensive the nearest neighbors' search will be, but in the real world, unstructured data does not live in isolation, and users expecting rich search experiences need to be able to search via a combination of structured and unstructured data. 

For example, users might arrive at your search interface with a brand preference in mind for the bikes dataset. Redis VSS queries can use this information to pre-filter the search space using a "primary filter query". In the following query definition, we pre-filter using the `brand` to consider only `Peaknetic` brand bikes. Notice that, before our primary filter query was `(*)`, AKA everything, but now we can narrow the search space using `(@brand:Peaknetic)` before the KNN query:

In [ ]:
hybrid_query = (
    Query('(@brand:Peaknetic)=>[KNN 3 @vector $query_vector AS vector_score]')
     .sort_by('vector_score')
     .return_fields('vector_score', 'id', 'brand', 'model', 'description')
     .dialect(2)
)

Filtering by the `Peaknetic` brand, for which there are 2 bikes in our collection, we can see the results returned for each of the query prompts. The query with the highest returned similarity score is "Comfortable commuter bike", followed by "Road bike for beginners". By filtering by brand, we fulfill the users' preferences and reduce the KNN search space by %80.

In [ ]:
create_query_table(hybrid_query, queries, encoded_queries)

#### Creating a VSS Range Query 

Range queries in Vector Similarity Search (VSS) involve retrieving items within a specific distance from a query vector. In this case, we consider "distance" to be the measure of similarity we've used to build our search indices; the smaller the distance, the more similar the items.

Let's say you want to find the bikes whose descriptions are within a certain distance from a query vector. We can use a range query to achieve this.
For example, the query command to return the top `4` documents within a `0.55` radius of a vectorized query would be as follows: 

```
1️⃣ FT.SEARCH idx:bikes_vss 
2️⃣   @vector:[VECTOR_RANGE $range $query_vector]=>{$YIELD_DISTANCE_AS: vector_score} 
3️⃣   SORTBY vector_score ASC
4️⃣   LIMIT 0 4 
5️⃣   DIALECT 2 
6️⃣   PARAMS 4 range 0.55 query_vector "\x9d|\x99>bV#\xbfm\x86\x8a\xbd\xa7~$?*...."
```

Where:

- 1️⃣ We use the `FT.SEARCH` command with our `idx:bikes_vss`.
- 2️⃣ and filter by the `vector` using the `VECTOR_RANGE` operator pasing the `$range` parameter, yield the vector distance between the vector field and the query result in a field named `vector_score`.
- 3️⃣ We sort the results by the yielded `vector_score`.
- 4️⃣ Limit the results to at most 4.
- 5️⃣ Once again, we set the RediSearch dialect to `2` to enable VSS functionality.
- 6️⃣ Finally we set the parameter values, `range` (`$range`) to `0.55` and the `query_vector` (`$query_vector`) to the encoded vectorized query. 

The equivalent Python query definition is shown below:

In [ ]:
range_query = (
    Query('@vector:[VECTOR_RANGE $range $query_vector]=>{$YIELD_DISTANCE_AS: vector_score}') 
    .sort_by('vector_score')
    .return_fields('vector_score', 'id', 'brand', 'model', 'description')
    .paging(0, 4)
    .dialect(2)
)

Let's run the first query prompt in our collection of queries, "Bike for small kids", using the VSS range query defined (`range_query`). We can use the `create_query_table` utility function to execute the query passing the extra parameter `$range` as a dictionary:

In [ ]:
create_query_table(range_query, queries[:1], encoded_queries[:1], {'range': 0.55})

The query returns two bikes in the specified range of our vectorized query, both with scores at or below `0.55`.

## Wrapping Up

In this guide, we learned how Redis, using the Redis Stack distribution, provides powerful search capabilities over structured and unstructured data. Redis support for vector data can enrich and enhance the user's search experience.
Although we focused on generating embeddings for unstructured data, the vector similarity approach can equally be employed with structure data, as long as a suitable vector generation technique is used.

The references below can help you learn more about Redis search capabilities:
* https://redis.io/docs/stack/search/
* https://redis.io/docs/stack/search/indexing_json/